In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_community.chat_models import ChatOllama


In [3]:
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages:Annotated[list[BaseMessage], add_messages]


In [4]:
llm=ChatOllama(model="mistral:latest",
    temperature=0.2)

def chat_node(state:ChatState):

    # take user uery state
    messages=state['messages']

    # send to llm
    response=llm.invoke(messages)

    # response store state
    return {'messages':[response]}





C:\Users\vikash.vk.kumar\AppData\Local\Temp\ipykernel_14308\2102244441.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm=ChatOllama(model="mistral:latest",


In [9]:
## add memory in loop of chat
from langgraph.checkpoint.memory import MemorySaver

In [10]:
checkpointer=MemorySaver()

graph=StateGraph(ChatState)
graph.add_node('chat_node',chat_node)
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

chatbot=graph.compile(checkpointer=checkpointer)




In [6]:
initial_state={
    'messages':[HumanMessage(content='What is capital of india')]
}
chatbot.invoke(initial_state)['messages'][1].content

" The capital of India is New Delhi. It serves as the center of the executive, legislative, and judicial branches of the Government of India. However, it's important to note that New Delhi is not one of the states of India, but a union territory and acts as the seat of the federal government. The largest city and commercial hub of India is Mumbai (formerly known as Bombay)."

In [13]:

thread_id="1"
while True:

    user_message= input('Type here:...... ')
    print('USer:', user_message)

    if user_message.strip().lower() in ['exit','quit','bye']:
        break

    config={'configurable':{'thread_id':thread_id}}

    response=chatbot.invoke({'messages':[HumanMessage(content=user_message)]}, config:config)

    print('AI: ', response['messages'][1].content)


SyntaxError: invalid syntax (3135279583.py, line 12)